In [41]:
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense
from tensorflow.python.keras.layers import deserialize, serialize
from tensorflow.python.keras.saving import saving_utils
import tempfile
import os
import pandas as pd
import numpy as np
import pickle as pk
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [42]:
# Define stoploss and take profit
stoploss = .05
takeprofit = .12
list_nPCs = [10, 15, 20, 25, 30]

In [43]:
# load the data
trainset_final = pd.read_csv('./Data/TrainSet_final_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))
trainset = pd.read_csv('./Data/TrainSet_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))

validation_set_final = pd.read_csv('./Data/ValidationSet_final_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))
validation_set = pd.read_csv('./Data/ValidationSet_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit))

In [44]:
# Hotfix function
def unpack(model, training_config, weights):
    restored_model = deserialize(model)
    if training_config is not None:
        restored_model.compile(
            **saving_utils.compile_args_from_training_config(
                training_config
            )
        )
    restored_model.set_weights(weights)
    return restored_model

def make_keras_picklable():
    def __reduce__(self):
        model_metadata = saving_utils.model_metadata(self)
        training_config = model_metadata.get("training_config", None)
        model = serialize(self)
        weights = self.get_weights()
        return (unpack, (model, training_config, weights))
    cls = Model
    cls.__reduce__ = __reduce__

# Run the function
make_keras_picklable()

In [45]:
# (b) Build and train several models different amounts of PCs
for nPCs in list_nPCs:
    print(nPCs)
    X = trainset_final.iloc[:, :nPCs]
    y = trainset["result"]

    # Build model and train it
    classifier = Sequential()
    #First Hidden Layer
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal', input_dim=nPCs))
    #Second, third and fourth  hidden Layers
    classifier.add(Dense(32, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))
    classifier.add(Dense(16, activation='relu', kernel_initializer='random_normal'))

    #Output Layer
    classifier.add(Dense(1, activation='sigmoid', kernel_initializer='random_normal'))
    #Compiling the neural network
    classifier.compile(optimizer ='adam',loss='binary_crossentropy', metrics =['accuracy'])
    #Fitting the data to the training dataset
    classifier.fit(X,y, batch_size=500, epochs=75, verbose =1)

    pk.dump(classifier, open("./Models/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit),"wb"))

10
Epoch 1/75
1/1 [==============================] - 1s 811ms/step - loss: 0.6932 - accuracy: 0.3092
Epoch 2/75
1/1 [==============================] - 0s 5ms/step - loss: 0.6929 - accuracy: 0.6958
Epoch 3/75
1/1 [==============================] - 0s 8ms/step - loss: 0.6927 - accuracy: 0.6958
Epoch 4/75
1/1 [==============================] - 0s 6ms/step - loss: 0.6924 - accuracy: 0.6958
Epoch 5/75
1/1 [==============================] - 0s 6ms/step - loss: 0.6922 - accuracy: 0.6958
Epoch 6/75
1/1 [==============================] - 0s 8ms/step - loss: 0.6920 - accuracy: 0.6958
Epoch 7/75
1/1 [==============================] - 0s 9ms/step - loss: 0.6917 - accuracy: 0.6958
Epoch 8/75
1/1 [==============================] - 0s 9ms/step - loss: 0.6914 - accuracy: 0.6958
Epoch 9/75
1/1 [==============================] - 0s 9ms/step - loss: 0.6912 - accuracy: 0.6958
Epoch 10/75
1/1 [==============================] - 0s 7ms/step - loss: 0.6909 - accuracy: 0.6958
Epoch 11/75
1/1 [=================

In [46]:
accuracies, nPCs_list = [], []
for nPCs in list_nPCs:
    print(nPCs)
    with open("./Models/DL_model_{}PC_stoploss{}_takeprofit{}.pkl".format(nPCs, stoploss, takeprofit), 'rb') as f:
        clf = pk.load(f)
    # Compute predictions on testset
    preds = (clf.predict(validation_set_final.iloc[:, :nPCs]) > 0.5)*1

    # Assess accuracy on Bullish predictions only (because we will only perform Bullish trades IRL) : we prioritize selectivity
    validation_set1 = validation_set[preds == 1].copy()
    accuracies.append(np.mean(preds == list(validation_set1['result'])))
    nPCs_list.append(nPCs)

10


/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


15
20


/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


25
30


/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/fromnumeric.py:3373: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/Users/jacobjalomo/opt/anaconda3/envs/algotrading/lib/python3.7/site-packages/numpy/core/_methods.py:170: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [48]:
recap = pd.DataFrame({'nPCs' : list(nPCs_list), 'Accuracy' : (list(accuracies))})
recap.to_csv('./Results/Comparative_All_models_stoploss{}_takeprofit{}.csv'.format(stoploss, takeprofit), index = False)
print(recap)

,nPCs,Accuracy
0,10,NaN
1,15,NaN
2,20,NaN
3,25,NaN
4,30,NaN
